# <font color="red">CMPT318 PROJECT</font>
MEMBER:
- JEFF
- 
- 

In [1]:
import pandas as pd
import numpy as np
from scipy import misc
from skimage.io import imread_collection
import re
import glob

## <font color="blue"> I. LOAD THE DATA </font>

### i. Load Webcam Images

In [2]:
image_collection = imread_collection('katkam-scaled/*.jpg')
images_np = np.array(image_collection)

In [3]:
# IMAGE IN NUMPY ARRAY.
# images_np
# images_np.shape

In [4]:
images_df = pd.DataFrame(
    images_np.reshape(images_np.shape[0],images_np.shape[1] * images_np.shape[2] * images_np.shape[3]))

In [5]:
# IMAGE IN PANDAS DATAFRAME, AND BEEN RESHAPED.
# images_df
# images_df.shape

## ii. Load Weather Observations Data

In [6]:
#reference: for load multiple file from forder: https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
weather_obv_filenames = glob.glob('yvr-weather/*.csv')
weather_obv_df = []

for filename in weather_obv_filenames:
    weather_obv_df.append(pd.read_csv(filename, skiprows=15)) # first 15 rows are general information, which is not useful data.

weather_obv_df = pd.concat(weather_obv_df).reset_index()
# pd.read_csv('weather-51442-201607.csv', skiprows=15)

In [7]:
# WEATHER OBSERVATIONS DATAFRAME.
# weather_obv_df

## <font color="blue">II. CLEAN&PREPARE DATA</font>

### i. Clean Webcam Images

In [8]:
# GET IMAGE FILE_NAME
image_filenames = np.array(image_collection.files)
# image_filenames[0]

In [9]:
# extract image shoot date form filename
re_image_date = re.compile(r'katkam-\d\d\d\d\d\d\d\d\d\d\d\d\d\d')
def get_image_time(path):
    matches = re_image_date.findall(path)
    if matches:
        # preocess match learned from https://docs.python.org/2/library/re.html
        # take the last match which will be file name
        result = matches[-1]
        return (int(result[-14:-10]), int(result[-10:-8]), int(result[-8:-6]), result[-6:-4] + ':' + result[-4:-2])
    else:
        return "wrong input file name format"
get_image_time = np.vectorize(get_image_time)
images_date = get_image_time(image_filenames)

In [10]:
# EXTRACTED IMAGE DATE
# images_date

In [11]:
# add relevant columns
images_df['Year'] = images_date[0]
images_df['Month'] = images_date[1]
images_df['Day'] = images_date[2]
images_df['Time'] = images_date[3]

In [12]:
# IMAGE DATA WITH DATE TIME
# images_df['Time']

## ii. Clean Weather Observations Data

In [13]:
# DROP UNCESSARY COLUMNS
cleaned_weather_obv = weather_obv_df.drop(['index','Data Quality'], axis=1)
cleaned_weather_obv = cleaned_weather_obv.drop(['Temp Flag', 'Stn Press Flag','Wind Chill Flag', 'Hmdx Flag', 'Visibility Flag', 'Wind Spd Flag', 'Wind Dir Flag', 'Rel Hum Flag', 'Dew Point Temp Flag'], axis=1)

# type(weather_obv_df['Year'][0])

In [14]:
# SPLIT cleaned_weather INTO TWO DATAFRAME, ONE WITH NaN WEATHER COLUMN, ONE WITH NOT NaN COLUMN
data_whoseWeather_IsNaN = cleaned_weather_obv[~ cleaned_weather_obv.Weather.notnull()] # weather with Nan
main_training_data = cleaned_weather_obv[cleaned_weather_obv.Weather.notnull()] # weather without Nan

In [15]:
# TWO SPLIT DATAFRAME
# data_whoseWeather_IsNaN
# main_training_data

In [16]:
main_training_data_withoutHW = main_training_data.drop(['Hmdx', 'Wind Chill'],axis=1)
final_data = main_training_data_withoutHW.dropna().copy()
# main_training_data = cleaned_data.copy()

In [17]:
# final_data

### weather description category include
- Clear: Clear, Mainly Clear
- Cloudy: Cloudy, Mostly Cloudy	
- Fog: Fog, Freezing Fog,
- Rain: Drizzle, Freezing Rain, Heavy Rain, Moderate Rain, Moderate Rain Showers, Rain, Rain Showers, Thunderstorms	
- Snow: Moderate Snow, Snow Pellets	,Ice Pellets, Snow Showers

<font color=red>Jacky留言：可以回头讨论 要怎么分类， 我暂时分成不止这5类， 回头商讨商讨</font>

In [18]:
# CLEAN HUMAN-GENERATED WEATHER DESCRIPTION
# SEE THE TOTAL VARIOUS WEATHER DESCRIPTION USE GROUPBY
# weather_category = final_data.groupby('Weather').count()
# weather_category

In [66]:
def removeDuplicate(lst):
    lst = lst.split(',')
#     print(lst)
    newlst = []
    for i in lst:
#         print (i)
        if i not in newlst:
            newlst.append(i)
    return newlst

In [67]:
# removeDuplicate("Rain,Clear,Rain,Rain,Fog")

In [68]:
re_clear = re.compile(r'Clear')
re_cloudy = re.compile(r'Cloudy')
re_fog = re.compile(r'Fog')
re_rain = re.compile(r'Rain')
re_snow = re.compile(r'Snow')
re_drizzle = re.compile(r'Drizzle')
re_thunderstorms = re.compile(r'Thunderstorms')
re_ice = re.compile(r'Ice')

def clean_weather_description(Str):
    result = ''
    match_clear = re_clear.search(Str)
    if match_clear:
        result = result + match_clear[0]+','
        
    match_cloudy = re_cloudy.search(Str)
    if match_cloudy:
        result = result + match_cloudy[0]+','
        
    match_fog = re_fog.search(Str)
    if match_fog:
        result = result + match_fog[0]+','
        
    match_rain = re_rain.search(Str)
    if match_rain:
        result = result + match_rain[0]+','
        
    match_snow = re_snow.search(Str)
    if match_snow:
        result = result + match_snow[0]+','
        
    match_drizzle = re_drizzle.search(Str)
    if match_drizzle:
        result = result + 'Rain'+','
        
    match_thunderstorms = re_thunderstorms.search(Str)
    if match_thunderstorms:
        result = result + 'Rain'+','
        
    match_ice = re_ice.search(Str)
    if match_ice:
        result = result + 'Snow'+','
    
    result = removeDuplicate(result)
        
    return result[:-1]

# clean_weather_description('Thunderstorms,Rain Showers')#test the function

In [69]:
# CLEAN THE WEATHER DESCRIPTION CATEGORY
final_data['Weather'] =final_data['Weather'].apply(clean_weather_description)

In [70]:
# final_data[final_data['Weather'] == 'Rain']  #1310 rows × 13 columns
# final_data['Weather'].to_csv('result_weather.csv')
# SHOW THE NEW WEATHER CATEGORY
# final_data
# weather_category = final_data.groupby('Weather').count()
# weather_category

## iii. Join Cleaned Webcam Image and Cleaned Weather obserbations Data Together

In [24]:
merged_data = final_data.merge(right = images_df, on = ['Year', 'Month', 'Day', 'Time'], how = 'inner')

In [25]:
#merged_data

In [26]:
#final_data

## <font color="blue"> III. ANALYSE THE DATA </font>

## <font color="black">try Bayesian Classifier</font>

In [84]:
# from sklearn.model_selection import train_test_split
X = merged_data.iloc[:,13:]
y = merged_data['Weather']
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y =mlb.fit_transform(y)
# bayes_X_train, bayes_X_test, bayes_y_train, bayes_y_test = train_test_split(X, y)
# from sklearn.naive_bayes import GaussianNB
# model = GaussianNB()
# model.fit(bayes_X_train, bayes_y_train)
# print(model.score(bayes_X_test, bayes_y_test))

In [85]:
# from scipy import stats
# print(stats.normaltest(bayes_X_train).pvalue)
# p-value too small no-normal
# don't use it

## <font color="black">SVC</font>

In [86]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)


In [30]:
# model = SVC(kernel='linear')
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test))

In [31]:
# model = SVC(kernel='rbf', decision_function_shape='ovr')     score = 0.397504456328   around 23min

In [32]:
# model = SVC(kernel='linear')        score = 0.668449197861   / 0.620320855615 ///0.641711229947

In [33]:
# model = SVC(kernel='rbf', decision_function_shape='ovr')   score = 0.623885918004

In [34]:
# e-1 model = SVC(kernel='linear', C=1e-1) 41   0.620320855615

In [35]:
# from sklearn.pipeline import make_pipeline
# from sklearn.decomposition import PCA
# model = make_pipeline(
#     PCA(1000),
#     SVC(kernel='linear', C=2.0)
# )
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test))   #0.614973262032/0.620320855615   37-39 2 min

In [36]:
# from sklearn.pipeline import make_pipeline
# from sklearn.decomposition import PCA
# model = make_pipeline(
#     PCA(1000),
#     SVC(kernel='linear', C=0.1)
# )
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test)) # little bit better 0.625668449198

In [37]:
# from sklearn.pipeline import make_pipeline
# from sklearn.decomposition import PCA
# model = make_pipeline(
#     PCA(100),
#     SVC(kernel='linear', C=2.0)
# )
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test)) #02

In [38]:
# from sklearn.pipeline import make_pipeline
# from sklearn.decomposition import PCA
# model = make_pipeline(
#     PCA(5000),
#     SVC(kernel='linear', C=2.0)
# )
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test)) #with in 2 min  refine pca   0.659536541889      0.670231729055    // change classify 0.63  0.654188948307 no too much difference 

## <font color="black">Nearest Neighbours</font>

In [39]:
# from sklearn.neighbors import KNeighborsClassifier
# model = KNeighborsClassifier(n_neighbors=10)
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test)) #4 min   0.609625668449 = neighbor5
# #3.40 min   0.600713012478   meighbors=10

## <font color="black">Jeff: I think if add weather condition into X, Scaler may need use. Because the unit is much different</font>

## <font color="black">neural_network</font>

In [40]:
# from sklearn.neural_network import MLPClassifier
# model = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(), random_state=0)
# model.fit(X_train, y_train)

# print(model.score(X_test, y_test))   # 1 min 0.393939393939

In [41]:
# from sklearn.neural_network import MLPClassifier
# model = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(10, 30),
#                       activation='logistic', random_state=0)
# model.fit(X_train, y_train)
# print(model.score(X_test, y_test)) #  1 min 0.399286987522

## <font color="black">add weather condition and time condition to X</font>

In [100]:
# X_condition

,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),0,1,2,...,147446,147447,147448,147449,147450,147451,147452,147453,147454,147455
0,18.9,15.6,81.0,30.0,22.0,48.3,101.31,89,121,180,...,0,0,0,0,0,0,0,0,0,0
1,21.2,15.8,71.0,30.0,27.0,48.3,101.28,125,150,191,...,0,0,0,0,0,0,0,0,0,0
2,24.0,15.9,60.0,29.0,22.0,40.2,101.20,131,160,200,...,0,0,0,0,0,0,0,0,0,0
3,23.7,15.6,60.0,29.0,12.0,48.3,101.04,183,210,239,...,0,0,0,0,0,0,0,0,0,0
4,22.0,15.7,67.0,20.0,7.0,48.3,100.96,140,148,151,...,0,0,0,0,0,0,0,0,0,0
5,19.4,15.7,79.0,30.0,13.0,48.3,101.27,119,151,198,...,0,0,0,0,0,0,0,0,0,0
6,20.7,14.9,69.0,29.0,25.0,48.3,101.28,85,114,174,...,0,0,0,0,0,0,0,0,0,0
7,22.0,15.2,65.0,26.0,12.0,48.3,101.20,138,164,189,...,0,0,0,0,0,0,0,0,0,0
8,23.9,12.7,49.0,23.0,8.0,48.3,101.03,157,189,228,...,0,0,0,0,0,0,0,0,0,0
9,19.8,14.5,71.0,25.0,12.0,48.3,100.92,115,122,132,...,0,0,0,0,0,0,0,0,0,0


In [99]:
X_condition = merged_data.drop(['Weather'], axis=1)
X_condition = X_condition.drop(['Time'], axis=1)
X_condition = X_condition.drop(['Date/Time','Year','Month','Day'], axis=1)
y_condition = merged_data['Weather']
X_condition_train, X_condition_test, y_condition_train, y_condition_test = train_test_split(X_condition, y_condition)
# X_condition

In [101]:
# X_condition['Weather']
from sklearn.pipeline import make_pipeline

from sklearn.decomposition import PCA
model = make_pipeline(
    PCA(5000),
    SVC(kernel='linear', C=2.0)
)
model.fit(X_condition_train, y_condition_train)
print(model.score(X_condition_test, y_condition_test))   #0.654188948307    no sightly different  0.655971479501

0.655971479501


## <font color="blue"> IV. PRESENT RESULT </font>

In [42]:
# 37-39